In [86]:
import load_data
import os
import pandas as pd
import sklearn.linear_model
import sklearn.ensemble
import matplotlib.pyplot as plt
import plot
import time
import datetime
import numpy as np

allDf = load_data.getPandasDataframes()

bejAirDf = allDf[('Beijing', 'air')]
bejMetDf = allDf[('Beijing', 'met')]
bejGrd = allDf[('Beijing', 'grid')]

bejAirDf['newTime'] = pd.to_datetime(bejAirDf['time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
bejMetDf['time'] = pd.to_datetime(bejMetDf['time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

bejMetDf = bejMetDf.rename(columns={'station_id':'met_station'})

bejDf = pd.DataFrame()
for key in ['station_id', 'PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration', 'CO_Concentration', 'O3_Concentration', 'SO2_Concentration']:
    bejDf[key] = bejAirDf[key]
bejDf['time'] = bejAirDf['newTime']

bejDf['hour'] = bejAirDf['newTime'].dt.hour
#bejDf['minute'] = bejAirDf['newTime'].dt.minute
#bejDf['second'] = bejAirDf['newTime'].dt.second
bejDf['day'] = bejAirDf['newTime'].dt.day
bejDf['month'] = bejAirDf['newTime'].dt.month
#bejDf['year'] = bejAirDf['newTime'].dt.year
bejDf['dayofweek'] = bejAirDf['newTime'].dt.dayofweek
bejDf['dayofweek'] = pd.Categorical(bejDf['dayofweek'], ordered=False)

valueDict = {(row['station_id'],row['time'].to_pydatetime()):row['PM10_Concentration'] for index, row in bejDf.iterrows()}
bejDf['PM10_3'] = bejDf['PM10_Concentration']
bejDf['PM10_3B'] = bejDf['PM10_Concentration']
bejDf['PM10_3C'] = bejDf['PM10_Concentration']
for index, row in bejDf.iterrows():
    rtime = row['time'].to_pydatetime()
    rtime = rtime - datetime.timedelta(hours=1)
    rtime2 = rtime - datetime.timedelta(hours=1)
    rtime3 = rtime2 - datetime.timedelta(hours=1)
    bejDf.at[index, 'PM10_3'] = valueDict.get((row['station_id'], rtime), np.NaN)
    bejDf.at[index, 'PM10_3B'] = valueDict.get((row['station_id'], rtime2), np.NaN)
    bejDf.at[index, 'PM10_3C'] = valueDict.get((row['station_id'], rtime3), np.NaN)


#stationDf = bejDf[bejDf['station_id']=='tiantan_aq']
#ToDo: limit to one station for now
#bejDf = stationDf

if False:
    plt.scatter(x=bejDf.PM25_Concentration, y=bejDf.PM10_Concentration)
    plt.xlabel("time", fontsize=14)
    plt.ylabel("PM10_Concentration", fontsize=14)
    f = plt.gcf()
    f.set_figheight(10)
    f.set_figwidth(15)
    plt.show()
    
# bejDf = bejDf.fillna(bejDf.median())
# bejDf = pd.get_dummies(bejDf, 'dum')


https://biendata.com/competition/airquality/bj/2017-01-01-0/2018-05-31-23/2k0d1d8
data/Beijing/air/bj_airquality_2017-01-01-0_2018-05-31-23.csv

https://biendata.com/competition/meteorology/bj/2017-01-01-0/2018-05-31-23/2k0d1d8
data/Beijing/met/bj_meteorology_2017-01-01-0_2018-05-31-23.csv

https://biendata.com/competition/meteorology/bj_grid/2017-01-01-0/2018-05-31-23/2k0d1d8
data/Beijing/grid/bj_grid_2017-01-01-0_2018-05-31-23.csv

https://biendata.com/competition/airquality/ld/2017-01-01-0/2018-05-31-23/2k0d1d8
data/London/air/ld_airquality_2017-01-01-0_2018-05-31-23.csv

https://biendata.com/competition/meteorology/ld/2017-01-01-0/2018-05-31-23/2k0d1d8
data/London/met/ld_meteorology_2017-01-01-0_2018-05-31-23.csv

https://biendata.com/competition/meteorology/ld_grid/2017-01-01-0/2018-05-31-23/2k0d1d8
data/London/grid/ld_grid_2017-01-01-0_2018-05-31-23.csv



In [541]:
# bejDf['businessHours'] = 0
# bejDf[(bejDf['hour'] >= 8) & (bejDf['hour']<=18)]['businessHours'] = 1
# bejDf.loc[workIndex,:]['businessHours'] = 1
# bejDf.head(n=100)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
bj1718meo = pd.read_csv('viz/stash/beijing_17_18_meo.csv')
bj1718meo = bj1718meo.rename(columns={'station_id':'met_station'})
bj_meo_stations = pd.read_csv('viz/stash/Beijing_meo_stations.csv')
bj_ring = pd.read_csv('viz/stash/Beijing_Neighbors.csv')
bj_nn = pd.read_excel('viz/stash/Beijing_Neighbors.xlsx')
bj_nn = bj_nn.rename(columns={"aq_station": "station_id"})

In [48]:
bj1718meo.head()

,met_station,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed,weather
0,shunyi_meo,116.615278,40.126667,2017-01-30 16:00:00,-1.7,1028.7,15,215.0,1.6,Sunny/clear
1,shunyi_meo,116.615278,40.126667,2017-01-30 17:00:00,-3.5,1028.4,24,16.0,1.0,Haze
2,shunyi_meo,116.615278,40.126667,2017-01-30 18:00:00,-3.7,1028.1,27,32.0,1.1,Haze
3,shunyi_meo,116.615278,40.126667,2017-01-30 19:00:00,-3.9,1027.6,27,21.0,0.9,Haze
4,shunyi_meo,116.615278,40.126667,2017-01-30 20:00:00,-4.1,1026.7,26,17.0,1.1,Haze


In [49]:
bj_meo_stations.head()

,station_id,longitude,latitude
0,shunyi_meo,116.615278,40.126667
1,hadian_meo,116.290556,39.986944
2,yanqing_meo,115.968889,40.449444
3,miyun_meo,116.864167,40.377500
4,huairou_meo,116.626944,40.357778


In [50]:
bj_nn.head()

,station_id,met_station
0,aotizhongxin_aq,beijing_grid_304
1,badaling_aq,beijing_grid_224
2,beibuxinqu_aq,beijing_grid_263
3,daxing_aq,beijing_grid_301
4,dingling_aq,beijing_grid_265


In [61]:
temp = bejDf.merge(bj_nn, on='station_id', how='left')

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


In [72]:
bjMinTime = min(bejDf.time)
bjMaxTime = max(bejDf.time)

In [ ]:
pd.merge(temp, bejMetDf,  how='left', left_on=['A_c1','c2']

,id,met_station,time,weather,temperature,pressure,humidity,wind_speed,wind_direction
0,2001610,shunyi_meo,2018-03-31 07:00:00,Sunny/clear,23.4,1006.9,19.0,3.3,212.0
1,2001611,hadian_meo,2018-03-31 07:00:00,Sunny/clear,23.7,1004.8,19.0,1.6,191.0
2,2001612,yanqing_meo,2018-03-31 07:00:00,Sunny/clear,25.8,952.5,14.0,5.4,247.0
3,2001613,miyun_meo,2018-03-31 07:00:00,Hail,23.9,1001.4,19.0,3.9,232.0
4,2001614,huairou_meo,2018-03-31 07:00:00,Sunny/clear,22.6,1001.3,19.0,3.2,243.0
5,2001615,shangdianzi_meo,2018-03-31 07:00:00,Sunny/clear,22.7,975.1,19.0,7.1,239.0
6,2001616,pinggu_meo,2018-03-31 07:00:00,Sunny/clear,23.2,1006.4,20.0,2.9,231.0
7,2001617,tongzhou_meo,2018-03-31 07:00:00,Hail,23.4,1008.0,19.0,1.2,205.0
8,2001618,chaoyang_meo,2018-03-31 07:00:00,Sunny/clear,23.2,1006.4,20.0,2.9,207.0
9,2001619,pingchang_meo,2018-03-31 07:00:00,Sunny/clear,23.5,1001.3,16.0,3.1,172.0


In [542]:
# ct = datetime.datetime.utcnow() - datetime.timedelta(days=2)
# splitDate = pd.datetime(ct.year,ct.month,ct.day,  ct.hour)
# trainDf = bejDf[bejDf['time'] < splitDate]
# testDf = bejDf[bejDf['time'] >= splitDate]

In [543]:
# cur = datetime.datetime.utcnow()
# current_time = pd.datetime(cur.year,cur.month,cur.day,cur.hour)
# d = list(testDf.time)
# date_list = [current_time - datetime.timedelta(hours=x) for x in range(0, 48)]
# date_list = [pd.to_datetime(i, format='%Y-%m-%d %H:%M:%S') for i in date_list]
# missingDatesFromTestDf = np.setdiff1d(sorted(date_list),list(set(sorted(d))))
# list(missingDatesFromTestDf)

[Timestamp('2018-04-28 05:00:00'),
 Timestamp('2018-04-29 00:00:00'),
 Timestamp('2018-04-29 16:00:00'),
 Timestamp('2018-04-29 17:00:00'),
 Timestamp('2018-04-29 23:00:00')]